In [ ]:
#@title Importing libraries (double-click to show code)
import os
import pkg_resources
pkg_resources.require("numpy>=`1.20.1")  #  to use specific numpy version
import numpy as np
import rsnapsim as rss
import sys
from sys import platform
from skimage import io ; from skimage.io import imread; from skimage.measure import find_contours
from random import randrange
import pandas as pd
import os; from os import listdir; from os.path import isfile, join
import re
import shutil
import pathlib
from pathlib import Path
from random import randrange
import ipywidgets as widgets ; from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox, Layout, GridspecLayout 
from IPython.display import Image
import dna_features_viewer
from dna_features_viewer import BiopythonTranslator, GraphicFeature, GraphicRecord, CircularGraphicRecord
import tqdm.notebook as tq

# Defining directories
current_dir = pathlib.Path().absolute()
sequences_dir = current_dir.parents[1].joinpath('DataBases','gene_files')
video_dir = current_dir.parents[1].joinpath('DataBases','videos_for_sim_cell')
rsnaped_dir = current_dir.parents[1].joinpath('rsnaped')

masks_dir = current_dir.parents[1].joinpath('DataBases','masks_for_sim_cell')

# Importing rSNAPsim_IP
sys.path.append(str(rsnaped_dir))
import rsnaped as rsp
import matplotlib

import matplotlib.pyplot as plt
plt.style.use('default')
plt.rcParams['figure.dpi'] = 120
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['axes.grid'] = False


In [ ]:
# Plasmid sequences
gene_file_pUB_SM_KDM5B_PP7 = str(sequences_dir.joinpath('pUB_SM_KDM5B_PP7.gb')) # plasmid pUB_SM_KDM5B_PP7 
gene_file_pUB_SM_p300_MS2 = str(sequences_dir.joinpath('pUB_SM_p300_MS2.gb')) # plasmid pUB_SM_p300_MS2

# Coding sequence
gene_file_KDM5B_PP7 = str(sequences_dir.joinpath('pUB_SM_KDM5B_PP7_coding_sequence.txt')) # coding sequence for SM_KDM5B_PP7    ### 5685 nt   ### 1895 codons
gene_file_p300_MS2 = str(sequences_dir.joinpath('pUB_SM_p300_MS2_coding_sequence.txt'))  # coding sequence for SM_p300_MS2      ### 8268 nt   ### 2756 codons

In [ ]:
from dna_features_viewer import BiopythonTranslator
class MyCustomTranslator(BiopythonTranslator):
    """Custom translator
    """
    def compute_feature_color(self, feature):
        if feature.type == "CDS":
            return "#57B956"
        elif feature.type == "FLAG":
            return "#57B956"
        elif feature.type == "MS2":
            return "#098BF5"
        elif feature.type == "PP7": 
            return "#EB5559"
        else:
            return "#C4B07B"

In [ ]:
graphic_record = MyCustomTranslator().translate_record(gene_file_pUB_SM_KDM5B_PP7) 
ax, _ = graphic_record.plot(figure_width=20, strand_in_label_threshold=7)
ax.set_title('pUB_SM_KDM5B_PP7 (1895 codons)')
graphic_record.plot_legend(ax=ax, loc=1, ncol=3, frameon=False)
#ax.figure.savefig("pUB_SM_KDM5B_PP7.png")

In [ ]:
graphic_record = MyCustomTranslator().translate_record(gene_file_pUB_SM_p300_MS2) 
ax, _ = graphic_record.plot(figure_width=20, strand_in_label_threshold=7)
ax.set_title('pUB_SM_p300_MS2 (2756 codons)')
graphic_record.plot_legend(ax=ax, loc=1, ncol=3, frameon=False)
#ax.figure.savefig("pUB_SM_p300_MS2.png")

In [ ]:
# Constant Parameters
save_as_tif = True # option to save the simulated video
save_dataframe = True # option to save the simulation output as a dataframe in format csv. 
store_videos_in_memory = True
save_as_gif = True
save_dataframe = True
scale_intensity_in_base_video= True
spot_size = 7 # size of spots in pixels
spot_sigma = 1 # sigma value for a 2d-gaussian point spread function.
number_cells = 1 # Number of simulated Cell
simulation_time_in_sec = 30
simulated_RNA_intensities_method = 'random_values'
frame_selection_empty_video = 'generate_from_gaussian' # Options are: 'constant' , 'shuffle' and 'loop' 'generate_from_gaussian'
basal_intensity_in_background_video = 10000

In [ ]:
list_gene_sequences = [gene_file_KDM5B_PP7,gene_file_p300_MS2] # path to gene sequences
list_label_names = [0,1] # list of strings used to generate a classification field in the output data frame
list_number_spots = [50, 50] # list of integers, where each element represents the number of spots
list_target_channels_proteins = [1, 2] # channel where the simulated protein spots will be located. Integer between 0 and 2. 
list_target_channels_mRNA = [0, 0] # channel where the simulated mRNA spots will be located. Integer between 0 and 2. 
list_diffusion_coefficients =[3, 3] # diffusion coefficients for each gene
list_elongation_rates = [10, 10] # elongation rates aa/sec
list_initiation_rates = [0.08, 0.08] # initiation rates 1/sec

In [ ]:
# list_gene_sequences = gene_file_KDM5B_PP7 # path to gene sequences
# list_label_names = 0 # list of strings used to generate a classification field in the output data frame
# list_number_spots = 50 # list of integers, where each element represents the number of spots
# list_target_channels_proteins = 1 # channel where the simulated protein spots will be located. Integer between 0 and 2. 
# list_target_channels_mRNA = 0 # channel where the simulated mRNA spots will be located. Integer between 0 and 2. 
# list_diffusion_coefficients = 3 # diffusion coefficients for each gene
# list_elongation_rates = 5.33 # elongation rates aa/sec
# list_initiation_rates = 0.08 # initiation rates 1/sec

In [ ]:
intensity_scale_ch0 = 2
intensity_scale_ch1 = 2
intensity_scale_ch2 = 2

In [ ]:
list_videos, _,merged_dataframe_simulated_cells, _,_,_,_ = rsp.simulate_cell( video_dir, 
                                                                        list_gene_sequences=list_gene_sequences,
                                                                        list_number_spots=list_number_spots,
                                                                        list_target_channels_proteins=list_target_channels_proteins,
                                                                        list_target_channels_mRNA=list_target_channels_mRNA, 
                                                                        list_diffusion_coefficients=list_diffusion_coefficients,
                                                                        list_elongation_rates=list_elongation_rates,
                                                                        list_initiation_rates=list_initiation_rates,
                                                                        list_label_names=list_label_names,
                                                                        masks_dir=masks_dir, 
                                                                        number_cells = number_cells,
                                                                        simulation_time_in_sec = simulation_time_in_sec,
                                                                        step_size_in_sec = 1,
                                                                        save_as_tif = save_as_tif, 
                                                                        save_as_gif = save_as_gif,
                                                                        save_dataframe = save_dataframe,
                                                                        frame_selection_empty_video=frame_selection_empty_video,
                                                                        spot_size = spot_size,
                                                                        spot_sigma = spot_sigma,
                                                                        intensity_scale_ch0 = intensity_scale_ch0,
                                                                        intensity_scale_ch1 = intensity_scale_ch1,
                                                                        intensity_scale_ch2 = intensity_scale_ch2,
                                                                        dataframe_format = 'long',
                                                                        simulated_RNA_intensities_method=simulated_RNA_intensities_method,
                                                                        store_videos_in_memory= True,
                                                                        scale_intensity_in_base_video =scale_intensity_in_base_video,
                                                                        basal_intensity_in_background_video=basal_intensity_in_background_video)

In [ ]:
merged_dataframe_simulated_cells.head()

In [ ]:
# Loading the simulated video
selected_cell = 0
simulated_video = list_videos[selected_cell]

In [ ]:
#@title Plotting the simulated cells
selected_time_point = 1
max_percentile=99.95
print('Dimensions in the simulated video: ', simulated_video.shape)
fig, ax = plt.subplots(3,2, figsize=(10, 7))
int_red = simulated_video[selected_time_point,:,:,0]
int_green = simulated_video[selected_time_point,:,:,1]
int_blue = simulated_video[selected_time_point,:,:,2]

print(np.min(int_red), np.max(int_red))
print(np.min(int_green), np.max(int_green))
print(np.min(int_blue), np.max(int_blue))

# Red
ax[0,0].imshow(int_red,cmap=plt.cm.Greys,vmax=np.percentile(int_red,max_percentile))
ax[0,0].set_title('Channel 0 = RNA');ax[0,0].set_xticks([]); ax[0,0].set_yticks([])
ax[0,1].hist(int_red.flatten(), bins=80,color='red')
ax[0,1].set_xlabel('Intensity'); ax[0,1].set_ylabel('Count')

# Green
ax[1,0].imshow(int_green,cmap=plt.cm.Greys,vmax=np.percentile(int_green,max_percentile))
ax[1,0].set_title('Channel 1 = Protein 1'); ax[1,0].set_xticks([]); ax[1,0].set_yticks([])
ax[1,1].hist(int_green.flatten(), bins=80,color='green')
ax[1,1].set_xlabel('Intensity'); ax[1,1].set_ylabel('Count')

# Blue
ax[2,0].imshow(int_blue,cmap=plt.cm.Greys,vmax=np.percentile(int_blue,max_percentile))
ax[2,0].set_title('Channel 2 = Protein 2'); ax[2,0].set_xticks([]); ax[2,0].set_yticks([])
ax[2,1].hist(int_blue.flatten(), bins=80,color='blue')
ax[2,1].set_xlabel('Intensity'); ax[2,1].set_ylabel('Count')
plt.show()

In [ ]:
df = merged_dataframe_simulated_cells
spot_type = 0
SNR_red_channel = df.loc[ (df['cell_number']==selected_cell) ].SNR_red.values
SNR_green_channel = df.loc[ (df['cell_number']==selected_cell) & (df['Classification']==0) ].SNR_green.values
SNR_blue_channel = df.loc[ (df['cell_number']==selected_cell)  &  (df['Classification']==1) ].SNR_blue.values


In [ ]:
#@title ####Plotting SNR

# Plotting
fig, ax = plt.subplots(1,3, figsize=(10, 3))
ax[0].hist(SNR_red_channel, bins = 40, color = 'orangered')
ax[0].set_ylabel('Count')
ax[0].set_title('$SNR_{Red}$')
ax[0].set_xlabel('SNR')

ax[1].hist(SNR_green_channel, bins = 40, color = 'limegreen')
ax[1].set_ylabel('Count')
ax[1].set_title('$SNR_{Green}$')
ax[1].set_xlabel('SNR')

ax[2].hist(SNR_blue_channel, bins = 40, color = 'blue')
ax[2].set_ylabel('Count')
ax[2].set_title('$SNR_{Blue}$')
ax[2].set_xlabel('SNR')
plt.subplots_adjust(wspace=0.5, hspace=0)

plt.show()

print('mean_SNR_red', np.median(SNR_red_channel))
print('mean_SNR_green', np.median(SNR_green_channel))
print('mean_SNR_blue', np.median(SNR_blue_channel))
